In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import time
import pathlib
project_path = pathlib.Path('.').absolute().parent
python_path = project_path/'src'
os.sys.path.insert(1, str(python_path))

In [ ]:
from tqdm.notebook import trange
import pybullet
import torch
    
import numpy as np
import pinocchio as pin

In [ ]:
from contact_mcts.objects import Cube
from contact_mcts.pvmcts import PolicyValueMCTS
from contact_mcts.trajectory import generate_random_poses
from contact_mcts.envs.fingers import FingerDoubleAndBox
from contact_mcts.pvnet import PolicyValueNet, ValueClassifier
from contact_mcts.params import get_default_params, update_params
from robot_properties_nyu_finger.config import NYUFingerDoubleConfig0, NYUFingerDoubleConfig1

## Set up parameters

In [ ]:
object_urdf = str(python_path/'contact_mcts'/'envs'/'resources'/'box.urdf')
robot_config = [NYUFingerDoubleConfig0(), NYUFingerDoubleConfig1()]
params = get_default_params(object_urdf, robot_config)

## Load networks

In [ ]:
# trained networks

device = torch.device('cpu')
pvnet = PolicyValueNet()
pvnet.load_state_dict(torch.load('../models/pvnet1.pt', 
                                 map_location=device))
value_classifier = ValueClassifier()
value_classifier.load_state_dict(torch.load('../models/value_classifier1.pt', 
                                            map_location=device))

# untrained networks
pvnet_untrained = PolicyValueNet()
value_classifier_untrained = ValueClassifier()

## Evaluate on random planar tasks

In [ ]:
# task specification
max_budget = 50
ntasks = 5
n_desired_poses = 2

# sampled region
lb = np.array([-0.08, -0.08, 0.1, 0., 0., -np.pi])
ub = np.array([ 0.08,  0.08, 0.1, 0., 0.,  np.pi])

diff_lb = np.array([-0.05, -0.05, 0, 0., 0., -np.pi / 4])
diff_ub = np.array([ 0.05,  0.05, 0, 0., 0.,  np.pi / 4])

In [ ]:
# store and compute the metrics
budget = []
errors = []
elapsed = []
failed_tasks = []

budget_untrained = []
errors_untrained = []
elapsed_untrained = []
failed_tasks_untrained = []

for task in trange(ntasks):
    desired_poses = generate_random_poses(n_desired_poses, lb, ub, diff_lb, diff_ub)    
    params = update_params(params, desired_poses)
    pose_init = pin.SE3ToXYZQUAT(params.desired_poses[0])
    box_pos = pose_init[:3]
    box_orn = pose_init[3:]
    env = FingerDoubleAndBox(params, box_pos, box_orn, server=pybullet.DIRECT)
    
    mcts = PolicyValueMCTS(params, env, 
                           networks=[pvnet, value_classifier])
    state = [[0, 0]]

    start = time.time()
    mcts.run(state, budget=max_budget, verbose=False)
    te = time.time() - start
    best_state, _ = mcts.get_solution()
    elapsed.append(te)
    budget.append(max_budget - mcts.budget)
        
    if best_state is not None:
        errors.append(mcts.reached_nodes_error[str(best_state)])
    else:
        failed_tasks.append(desired_poses)

    mcts_untrained = PolicyValueMCTS(params, env, 
                                     networks=[pvnet_untrained, value_classifier_untrained])
    state = [[0, 0]]

    start = time.time()
    mcts_untrained.run(state, budget=max_budget, verbose=False)
    te_untrained = time.time() - start
    best_state, _ = mcts_untrained.get_solution()
    elapsed_untrained.append(te_untrained)
    budget_untrained.append(max_budget - mcts_untrained.budget)
        
    if best_state is not None:
        errors_untrained.append(mcts_untrained.reached_nodes_error[str(best_state)])
    else:
        failed_tasks_untrained.append(desired_poses)
        
    env.close()

In [ ]:
print(len(errors_untrained)/ntasks)
print(np.mean(errors_untrained, axis=0))
print(np.max(errors_untrained, axis=0))

print(np.mean(budget_untrained))
print(np.max(budget_untrained))
print(np.mean(elapsed_untrained))
print(np.max(elapsed_untrained))

In [ ]:
print(len(errors)/ntasks)
print(np.mean(errors, axis=0))
print(np.max(errors, axis=0))

print(np.mean(budget))
print(np.max(budget))
print(np.mean(elapsed))
print(np.max(elapsed))